In [1]:
%run sources/slicer.py
%run sources/time_series_to_intervals.py
%run sources/time_series_event_merge.py

main_value = "p02" #variable
main_feature = "participant"


In [2]:
feature = "distance"
p = pd.read_csv(f"processed_data/fitbit/{feature}.csv")
p[p[main_feature] == main_value][feature].describe()

count    107326.000000
mean       1637.074139
std        2921.116056
min           0.000000
25%           0.000000
50%           0.000000
75%        1860.000000
max       19907.000000
Name: distance, dtype: float64

In [5]:
time_feature = "vt"
"""
timeseries_df_list = {"paths": ["processed_data/fitbit/calories.csv",
                                "processed_data/fitbit/distance.csv",
                                "processed_data/fitbit/steps.csv",
                                "processed_data/fitbit/heart_rate.csv"],
                       "group_features": ["calories", "distance", "steps", "bpm"],
                       "bucket_values": [[1.2, 1.57], [1860], [26], [46, 53, 62]]} #variable
"""
timeseries_df_list = {"paths": [],
                       "group_features": [],
                       "bucket_values": []}

event_df_list = {"paths": ["processed_data/fitbit/exercise.csv",
                            "processed_data/fitbit/sleep_score.csv",
                            "processed_data/fitbit/sleep.csv"],
                  "group_features": ["Activity", "label", "sleep_phase"],
                  "value_feature": ["value", "value", "value"]}

slicer_params = {"stride": 24, "ow": 20, "ww": 4, "pw": 48, "granularity": "hours"} #variable

final_odf, final_pdf = samples_from_dataset(main_value, main_feature, time_feature, timeseries_df_list, event_df_list, slicer_params)

100%|██████████| 143/143 [00:00<00:00, 1618.64it/s]


In [6]:
final_odf[final_odf.i == 6]

,i,vt,label,value
171,6,11998.0,Walk,3379000
172,6,29279.0,light,90
173,6,29369.0,wake,570
174,6,29939.0,light,630
175,6,30569.0,rem,720
176,6,31289.0,wake,300
177,6,31589.0,light,900
178,6,32489.0,deep,1680
179,6,34169.0,light,1920
180,6,36089.0,rem,450


In [7]:
def sleep_score_from_prediction_window(dfs):
    r,ids =[],[]
    for df in dfs["label"]:
        scs = df.to_dict(orient="records")
        scs.sort(key=lambda x: x['vt'])
        if len(df) > 0:
            r.append(scs[0]['value']) 
            ids.append(scs[0]['i'])   
    return pd.DataFrame({'SleepScore':r, 'i':ids})




In [8]:
def df_to_sequences(df):
    df["sequence"] = df.apply(lambda row: (row['vt'], row['label'], row['value']), axis=1)
    r =df.groupby("i")["sequence"].agg(list).reset_index()
    for i in range(len(r)):
        r.iloc[i]["sequence"].sort(key=lambda x: x[0])
    return r

Z = df_to_sequences(final_odf)

In [9]:
Z = pd.merge(Z, sleep_score_from_prediction_window(final_pdf), on='i', how='inner')

In [10]:
Z["SleepScore"].describe()

count    141.000000
mean      71.219858
std        6.489654
min       57.000000
25%       67.000000
50%       72.000000
75%       76.000000
max       83.000000
Name: SleepScore, dtype: float64

In [11]:
def score_buckets(s, buckets):
    for i in range(len(buckets)):
        if s <= buckets[i]:
            if i == 0 or s > buckets[i-1]:
                return i 
    return len(buckets)    

scores = [67,73,76] #variable

Z["class"] = Z["SleepScore"].apply(lambda x: score_buckets(x, scores))

In [12]:
Z

,i,sequence,SleepScore,class
0,0,"[(0.0, Walk, 922000), (6930.0, Run, 4315000), ...",72,1
1,1,"[(232.0, Walk, 1280000), (10012.0, Sport, 1792...",67,0
2,2,"[(1388.0, Run, 3789000), (34229.0, wake, 30), ...",80,3
3,3,"[(35609.0, wake, 30), (35639.0, light, 1140), ...",79,3
4,4,"[(29249.0, wake, 600), (29849.0, light, 30), (...",59,0
...,...,...,...,...
136,138,"[(29969.0, light, 1470), (31439.0, deep, 1380)...",74,2
137,139,"[(30629.0, wake, 30), (30659.0, light, 1050), ...",70,1
138,140,"[(25019.0, wake, 30), (25049.0, light, 1230), ...",65,0
139,141,"[(30029.0, wake, 300), (30329.0, light, 690), ...",62,0


In [13]:
import json
from datetime import datetime

In [14]:
ZPARAMS = {
    "main_value":main_value, 
    "main_feature":main_feature, 
    "time_feature":time_feature, 
    "timeseries_df_list":timeseries_df_list, 
    "event_df_list":event_df_list, 
    "slicer_params":slicer_params,
    "class":{"SleepScore": scores}
}
TIME = datetime.now().strftime('%Y%m%d_%H%M')
ZNAME = f"{TIME}_fitbit_{main_value}_events"


In [15]:
Z[["sequence", "class"]].to_csv(f"datasets/{ZNAME}.csv", index=False)
with open(f"datasets/{ZNAME}.json", 'w') as json_file:
    json.dump(ZPARAMS, json_file, indent=2)
